# Classification d'images (MNIST) avec Keras (MLP)

## Imports




In [12]:
!pip install tensorflow

In [13]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
import time

## Chargement de la base MNIST

La première étape consiste à charger les données MNIST selon la documentation de keras https://keras.io/datasets/

In [14]:
# Configuration options
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Load the data
x_train.shape

(60000, 28, 28)

## Prétraitement des données: l'entrée
- normaliser les données (pour que l'entrée  soit dans [0,1])
- remodeler les données d'entrée de sorte que les entrées `x` soient des vecteurs de taille 784, adaptés pour les RNN keras et non des images (matrices 28 x 28)


In [15]:
# Convert into greyscale

x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Reshape the data - MLPs do not understand such things as '2D'.
# Reshape to 28 x 28 pixels = 784 features

x_train_flat = x_train.reshape(-1, 784)
x_train_flat.shape

x_test_flat = x_test.reshape(-1, 784)
x_test_flat.shape

(10000, 784)

## Convertir les classes cibles en categories (et non numerique)
- utiliser la fonction Keras `to_categorical()`


In [16]:
# Convertir les target classes to categorical ones
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

## Développer un MLP

Commençons par les réseaux de neurones entièrement connectés (NN) également appelés perceptrons multicouches (MLP).
Comme lors des TP précédents, le modèle séquentiel de keras sera utilisé avec des couches `Dense`

### Premier essai
Vous allez essayer de reproduire LeCun et al. 1998 dont les resultats sont disponibles sur le site Web du MNIST (http://yann.lecun.com/exdb/mnist/).
le taux d'erreur signalé est de 4,7%.
Nous commençons par créer l'architecture du MLP.
   - dans ce cas le réseau de neurones est entièrement connecté: le NN est séquentiel,
   - la première couche contient 300 neurones avec "relu" comme fonction d'activation,
   - la couche de sortie contient 10 neurones (un par classe), la fonction d'activation est: `softmax`.

In [17]:
# Set the input shape
input_shape = (784,)

model = Sequential()

model.add(Dense(300, activation='relu', input_shape=input_shape))

model.add(Dense(10, activation='softmax'))

# Créez un model Séquentiel (voir documentation Keras)

model = Sequential()

model.add(Dense(300, activation="relu", input_shape=(784,)))
model.add(Dense(10, activation="softmax"))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 300)               235500    
                                                                 
 dense_7 (Dense)             (None, 10)                3010      
                                                                 
Total params: 238,510
Trainable params: 238,510
Non-trainable params: 0
_________________________________________________________________


Maintenant, vous aller définir les parametres d'apprentissage du modèle 
- La première étape ici est de définir la fonction de coût (`loss`) et l'optimiseur (`optimizer`).
    - Un choix naturel pour la fonction de perte `loss` est «catégorical_crossentropy» bien adaptée à la classification multiclasse. (La même vue dans le TP1)
    - pour l'optimiseur, choisissez une descente de gradient stochastique simple (`SGD()`).
    
    

- Lancer l'apprentissage et enregistrer le temps d'apprentissage

In [18]:
# Compilez le model et commencez l'apprentissage (avec 10 itérations)

loss = 'categorical_crossentropy'

optimizer = SGD()

start = time.time()

model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

history = model.fit(x_train_flat, y_train, epochs=10, batch_size=128)

end = time.time()

print("Temps d'exécution: ", end - start)

history.history

Epoch 1/10
469/469 [==============================] - 1s 2ms/step - loss: 1.1283 - accuracy: 0.7498
Epoch 2/10
469/469 [==============================] - 1s 2ms/step - loss: 0.5419 - accuracy: 0.8688
Epoch 3/10
469/469 [==============================] - 3s 7ms/step - loss: 0.4344 - accuracy: 0.8864
Epoch 4/10
469/469 [==============================] - 1s 3ms/step - loss: 0.3856 - accuracy: 0.8958
Epoch 5/10
469/469 [==============================] - 1s 2ms/step - loss: 0.3560 - accuracy: 0.9029
Epoch 6/10
469/469 [==============================] - 1s 2ms/step - loss: 0.3349 - accuracy: 0.9076
Epoch 7/10
469/469 [==============================] - 1s 2ms/step - loss: 0.3186 - accuracy: 0.9115
Epoch 8/10
469/469 [==============================] - 1s 2ms/step - loss: 0.3050 - accuracy: 0.9152
Epoch 9/10
469/469 [==============================] - 1s 2ms/step - loss: 0.2934 - accuracy: 0.9186
Epoch 10/10
469/469 [==============================] - 1s 2ms/step - loss: 0.2833 - accuracy: 0.9213

{'loss': [1.1282881498336792,
  0.5419073700904846,
  0.4343818426132202,
  0.38560807704925537,
  0.3560313582420349,
  0.3348805904388428,
  0.318589985370636,
  0.3050137758255005,
  0.2933981120586395,
  0.28330135345458984],
 'accuracy': [0.7498499751091003,
  0.8688499927520752,
  0.8864499926567078,
  0.8957833051681519,
  0.9028833508491516,
  0.9076166749000549,
  0.9114500284194946,
  0.9151999950408936,
  0.9185500144958496,
  0.9212833046913147]}

   - Mesurer le taux d'erreur sur la base `test`

In [26]:
# Evaluez votre model après l'apprentissage sur la base donnée Test

start = time.time()

score = model.evaluate(x_test_flat, y_test, verbose=0)

end = time.time()

# Print Test Loss et Test Accuracy de votre modèle

print("Test loss:", score[0])
print("Test accuracy:", score[1])
print("Time : ", end - start)

Test loss: 0.2686576247215271
Test accuracy: 0.9265000224113464
Time :  0.39216113090515137


- Visualisation des résultats d'apprentissage et de test

In [20]:
# Visualisez les courbes (historique des données d'apprentissage et de test) 


### Deuxième essai
Essayons d'ameliorer un peu l'apprentissage en introduisant un peu de punch dans notre optimiseur.
Pour ce faire, nous ajoutons un terme d'élan (`momentum = 0.9`) et une pénalité L2 (` decay = 1e-6`).

Cela se fait en remplaçant l'instruction `
optimiseur = SGD (), `
par
`
optimizer = SGD (learning_rate = 0.01, decay = 1e-6, momentum = 0.9, nesterov = False), `

    
  - Relancer l'apprentissage sur ce modele et enregistrer le temps d'apprentissage



In [28]:
# Mêmes étapes de création de modèle (nommez-le différemment), modifiez que l'optimiseur.

modelBis = Sequential()

modelBis.add(Dense(300, activation="relu", input_shape=(784,)))
modelBis.add(Dense(10, activation="softmax"))

modelBis.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 300)               235500    
                                                                 
 dense_9 (Dense)             (None, 10)                3010      
                                                                 
Total params: 238,510
Trainable params: 238,510
Non-trainable params: 0
_________________________________________________________________


   - Mesurer le taux d'erreur sur la base `test`

In [29]:
# Compilez le model et commencez l'apprentissage (avec 10 itérations)

loss = 'categorical_crossentropy'

optimizer = SGD(learning_rate = 0.01, decay = 1e-6, momentum = 0.9, nesterov = False)

start = time.time()

modelBis.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

history = modelBis.fit(x_train_flat, y_train, epochs=10, batch_size=128)

end = time.time()

print("Temps d'exécution: ", end - start)

history.history

Epoch 1/10
469/469 [==============================] - 1s 2ms/step - loss: 0.4756 - accuracy: 0.8683
Epoch 2/10
469/469 [==============================] - 1s 2ms/step - loss: 0.2523 - accuracy: 0.9292
Epoch 3/10
469/469 [==============================] - 1s 2ms/step - loss: 0.2006 - accuracy: 0.9441
Epoch 4/10
469/469 [==============================] - 1s 2ms/step - loss: 0.1685 - accuracy: 0.9517
Epoch 5/10
469/469 [==============================] - 1s 2ms/step - loss: 0.1456 - accuracy: 0.9592
Epoch 6/10
469/469 [==============================] - 1s 2ms/step - loss: 0.1282 - accuracy: 0.9651
Epoch 7/10
469/469 [==============================] - 1s 2ms/step - loss: 0.1143 - accuracy: 0.9682
Epoch 8/10
469/469 [==============================] - 1s 2ms/step - loss: 0.1037 - accuracy: 0.9717
Epoch 9/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0945 - accuracy: 0.9736
Epoch 10/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0864 - accuracy: 0.9767

{'loss': [0.4755588173866272,
  0.2523110806941986,
  0.20058713853359222,
  0.1684914380311966,
  0.14564384520053864,
  0.12815560400485992,
  0.11433720588684082,
  0.10369344055652618,
  0.09447868168354034,
  0.08644025772809982],
 'accuracy': [0.8682666420936584,
  0.9291666746139526,
  0.9441166520118713,
  0.9517333507537842,
  0.9592000246047974,
  0.9651166796684265,
  0.9682333469390869,
  0.9717166423797607,
  0.9735666513442993,
  0.9767000079154968]}

In [30]:
# Evaluez votre model après l'apprentissage sur la base donnée Test

start = time.time()

score = modelBis.evaluate(x_test_flat, y_test, verbose=0)

end = time.time()

# Print Test Loss et Test Accuracy de votre modèle

print("Test loss:", score[0])
print("Test accuracy:", score[1])
print("Time : ", end - start)

Test loss: 0.09588633477687836
Test accuracy: 0.9718999862670898
Time :  0.45223522186279297


- Comparer les temps d'apprentissage et les taux d'erreur. Que remarquez-vous ?

Premier modèle (**model**):

    - Test loss: 0.2686576247215271
    - Test accuracy: 0.9265000224113464
    - Time :  0.39216113090515137

Second modèle (**modelBis**):

    - Test loss: 0.09588633477687836
    - Test accuracy: 0.9718999862670898
    - Time :  0.45223522186279297

On voit que nous avons moins environ deux fois moins d'erreurs sur notre second modèle et que la précision est meilleure avec le second également.
En revanche, le temps d'exécution est un peu plus long pour le second modèle.

In [25]:
# Visualisez les courbes (historique des données d'apprentissage et de test) 